In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['gendered-pronoun-resolution', 'gap-coreference']


In [2]:
import zipfile
import gc
from tqdm import tqdm_notebook as tqdm

!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
with zipfile.ZipFile("uncased_L-12_H-768_A-12.zip","r") as zip_ref:
    zip_ref.extractall()
!ls 'uncased_L-12_H-768_A-12'

--2019-03-18 11:43:31--  https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.20.128, 2607:f8b0:400e:c07::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.20.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 407727028 (389M) [application/zip]
Saving to: ‘uncased_L-12_H-768_A-12.zip’

uncased_L-12_H-768_ 100%[===================>] 388.84M   139MB/s    in 2.8s    

2019-03-18 11:43:34 (139 MB/s) - ‘uncased_L-12_H-768_A-12.zip’ saved [407727028/407727028]

bert_config.json		     bert_model.ckpt.index  vocab.txt
bert_model.ckpt.data-00000-of-00001  bert_model.ckpt.meta


In [3]:
!wget https://raw.githubusercontent.com/google-research/bert/master/modeling.py 
!wget https://raw.githubusercontent.com/google-research/bert/master/extract_features.py 
!wget https://raw.githubusercontent.com/google-research/bert/master/tokenization.py

--2019-03-18 11:43:43--  https://raw.githubusercontent.com/google-research/bert/master/modeling.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 37922 (37K) [text/plain]
Saving to: ‘modeling.py’

modeling.py         100%[===================>]  37.03K  --.-KB/s    in 0.007s  

2019-03-18 11:43:43 (4.91 MB/s) - ‘modeling.py’ saved [37922/37922]

--2019-03-18 11:43:44--  https://raw.githubusercontent.com/google-research/bert/master/extract_features.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13898 (14K) [text/plain]
Saving to: ‘extrac

In [4]:
import modeling
import extract_features
import tokenization
import tensorflow as tf

In [5]:
test_df  = pd.read_table('../input/gap-coreference/gap-development.tsv')
train_df = pd.read_table('../input/gap-coreference/gap-test.tsv')
val_df   = pd.read_table('../input/gap-coreference/gap-validation.tsv')
train_df.head()

,ID,Text,Pronoun,Pronoun-offset,A,A-offset,A-coref,B,B-offset,B-coref,URL
0,test-1,Upon their acceptance into the Kontinental Hoc...,His,383,Bob Suter,352,False,Dehner,366,True,http://en.wikipedia.org/wiki/Jeremy_Dehner
1,test-2,"Between the years 1979-1981, River won four lo...",him,430,Alonso,353,True,Alfredo Di St*fano,390,False,http://en.wikipedia.org/wiki/Norberto_Alonso
2,test-3,Though his emigration from the country has aff...,He,312,Ali Aladhadh,256,True,Saddam,295,False,http://en.wikipedia.org/wiki/Aladhadh
3,test-4,"At the trial, Pisciotta said: ``Those who have...",his,526,Alliata,377,False,Pisciotta,536,True,http://en.wikipedia.org/wiki/Gaspare_Pisciotta
4,test-5,It is about a pair of United States Navy shore...,his,406,Eddie,421,True,Rock Reilly,559,False,http://en.wikipedia.org/wiki/Chasers


In [6]:
def count_char(text, offset):   
    count = 0
    for pos in range(offset):
        if text[pos] != " ": count +=1
    return count

def candidate_length(candidate):
    count = 0
    for i in range(len(candidate)):
        if candidate[i] !=  " ": count += 1
    return count

def count_token_length_special(token):
    count = 0
    special_token = ["#", " "]
    for i in range(len(token)):
        if token[i] not in special_token: count+=1
    return count

def embed_by_bert(df):
    
    text = df['Text']
    text.to_csv('input.txt', index=False, header=False)
    os.system("python3 extract_features.py \
               --input_file=input.txt \
               --output_file=output.jsonl \
               --vocab_file=uncased_L-12_H-768_A-12/vocab.txt \
               --bert_config_file=uncased_L-12_H-768_A-12/bert_config.json \
               --init_checkpoint=uncased_L-12_H-768_A-12/bert_model.ckpt \
               --layers=-1 \
               --max_seq_length=256 \
               --batch_size=8")
    
    bert_output = pd.read_json("output.jsonl", lines = True)
    bert_output.head()
    
    os.system("rm input.txt")
    os.system("rm output.jsonl")
    
    index = df.index
    columns = ["emb_A", "emb_B", "emb_P", "label"]
    emb = pd.DataFrame(index = index, columns = columns)
    emb.index.name = "ID"
    
    for i in tqdm(range(len(text))):
        
        features = bert_output.loc[i, "features"]
        P_char_start = count_char(df.loc[i, 'Text'], df.loc[i, 'Pronoun-offset'])
        A_char_start = count_char(df.loc[i, 'Text'], df.loc[i, 'A-offset'])
        B_char_start = count_char(df.loc[i, 'Text'], df.loc[i, 'B-offset'])
        A_length = candidate_length(df.loc[i, 'A'])
        B_length = candidate_length(df.loc[i, 'B'])
        
        emb_A = np.zeros(768)
        emb_B = np.zeros(768)
        emb_P = np.zeros(768)
        
        char_count = 0
        cnt_A, cnt_B = 0, 0
        
        for j in range(2, len(features)):
            token = features[j]["token"]
            token_length = count_token_length_special(token)
            if char_count == P_char_start:
                emb_P += np.asarray(features[j]["layers"][0]['values']) 
            if char_count in range(A_char_start, A_char_start+A_length):
                emb_A += np.asarray(features[j]["layers"][0]['values'])
                cnt_A += 1
            if char_count in range(B_char_start, B_char_start+B_length):
                emb_B += np.asarray(features[j]["layers"][0]['values'])
                cnt_B += 1                
            char_count += token_length
        
        emb_A /= cnt_A
        emb_B /= cnt_B
        
        label = "Neither"
        if (df.loc[i,"A-coref"] == True):
            label = "A"
        if (df.loc[i,"B-coref"] == True):
            label = "B"

        emb.iloc[i] = [emb_A, emb_B, emb_P, label]
        
    return emb     

In [7]:
test_emb = embed_by_bert(test_df)
test_emb.to_json("contextual_embeddings_gap_test.json", orient = 'columns')

validation_emb = embed_by_bert(val_df)
validation_emb.to_json("contextual_embeddings_gap_validation.json", orient = 'columns')

train_emb = embed_by_bert(train_df)
train_emb.to_json("contextual_embeddings_gap_train.json", orient = 'columns')

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:66: RuntimeWarning: invalid value encountered in true_divide
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:67: RuntimeWarning: invalid value encountered in true_divide


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:67: RuntimeWarning: invalid value encountered in true_divide


In [8]:
import spacy
nlp = spacy.load('en_core_web_lg')

def bs(lens, target):
    low, high = 0, len(lens) - 1

    while low < high:
        mid = low + int((high - low) / 2)

        if target > lens[mid]:
            low = mid + 1
        elif target < lens[mid]:
            high = mid
        else:
            return mid + 1

    return low

def b_distance(dist):
    
    buckets = [1, 2, 3, 4, 5, 8, 16, 32, 64]  
    low, high = 0, len(buckets)
    while low < high:
        mid = low + int((high-low) / 2)
        if dist > buckets[mid]:
            low = mid + 1
        elif dist < buckets[mid]:
            high = mid
        else:
            return mid

    return low

def distance_features(char_offsetP, char_offsetA, char_offsetB, text):
    
    doc = nlp(text)
    lens = [token.idx for token in doc]
    mention_offsetP = bs(lens, char_offsetP) - 1
    mention_offsetA = bs(lens, char_offsetA) - 1
    mention_offsetB = bs(lens, char_offsetB) - 1
    mention_distA = mention_offsetP - mention_offsetA
    mention_distB = mention_offsetP - mention_offsetB
    
    dist_binA = b_distance(mention_distA)
    dist_binB = b_distance(mention_distB)
    
    return dist_binA, dist_binB

def extract_dist_features(df):
    
    index = df.index
    columns = ["D_PA", "D_PB"]
    dist_df = pd.DataFrame(index = index, columns = columns)

    for i in tqdm(range(len(df))):
        text = df.loc[i, 'Text']
        P_offset = df.loc[i,'Pronoun-offset']
        A_offset = df.loc[i, 'A-offset']
        B_offset = df.loc[i, 'B-offset']   
        
        dist_df.iloc[i] = distance_features(P_offset, A_offset, B_offset, text)
    
    return dist_df

In [9]:
test_dist_df = extract_dist_features(test_df)
test_dist_df.to_csv('test_dist_df.csv', index=False)
val_dist_df = extract_dist_features(val_df)
val_dist_df.to_csv('val_dist_df.csv', index=False)
train_dist_df = extract_dist_features(train_df)
train_dist_df.to_csv('train_dist_df.csv', index=False)

In [10]:
from keras.layers import *
import keras.backend as K
from keras.models import *

class End2End_NCR():
    
    def __init__(self, word_input_shape, dist_shape, dist_embed_dim=20):
        
        self.word_input_shape = word_input_shape
        self.dist_shape = dist_shape
        self.buckets = [1, 2, 3, 4, 5, 8, 16, 32, 64] 
        self.dist_embed_dim = dist_embed_dim
        self.hidden_dim = 120
        
    def build(self):
        
        P, A, B = Input((self.word_input_shape,)), Input((self.word_input_shape,)), Input((self.word_input_shape,))
        dist_PA, dist_PB = Input((self.dist_shape,)), Input((self.dist_shape,))
        
        self.dist_embed   = Embedding(len(self.buckets)+1, self.dist_embed_dim)
        self.element_wise = Multiply()
        self.concat       = Concatenate(axis=-1)
        self.flatten      = Flatten()
        self.add          = Add()
        
        dist_PA_embed = self.dist_embed(dist_PA)
        dist_PA_embed = self.flatten(dist_PA_embed)
        dist_PB_embed = self.dist_embed(dist_PB)
        dist_PB_embed = self.flatten(dist_PB_embed)
        
        #Scoring
        score_P = self.FFNN1(P)
        score_A = self.FFNN1(A)
        score_B = self.FFNN1(B)   
        element_wisePA = self.element_wise([P, A])
        element_wisePB = self.element_wise([P, B])        
        ante_scorePA   = self.FFNN2(self.concat([P, A, element_wisePA, dist_PA_embed]))
        ante_scorePB   = self.FFNN2(self.concat([P, B, element_wisePB, dist_PB_embed]))
        score_PA = self.add([score_P, score_A, ante_scorePA]) #(batch_size, 1)
        score_PB = self.add([score_P, score_B, ante_scorePB]) #(batch_size, 1)
        score_e  = Lambda(lambda x: K.zeros_like(x))(score_PB)
        
        #Final Output
        output = self.concat([score_PA, score_PB, score_e])
        output = Activation('softmax')(output)
        
        model = Model([P, A, B, dist_PA, dist_PB], output)
        
        return model
        
    def FFNN1(self, x):
        
        x = Dense(self.hidden_dim, activation='relu', use_bias=True)(x)
        x = Dropout(rate=0.2)(x)
        x = Dense(self.hidden_dim, activation='relu', use_bias=True)(x)
        x = Dropout(rate=0.2)(x)
        x = Dense(1, activation='linear')(x)
        
        return x
    
    def FFNN2(self, x):
        
        x = Dense(self.hidden_dim, activation='relu', use_bias=True)(x)
        x = Dropout(rate=0.2)(x)
        x = Dense(self.hidden_dim, activation='relu', use_bias=True)(x)
        x = Dropout(rate=0.2)(x)
        x = Dense(1, activation='linear')(x)
        
        return x

Using TensorFlow backend.


In [11]:
def create_input(embed_df, dist_df):
    
    assert len(embed_df) == len(dist_df)
    all_P, all_A, all_B = [] ,[] ,[]
    all_label = []
    all_dist_PA, all_dist_PB = [], []
    
    for i in tqdm(range(len(embed_df))):
        
        all_P.append(embed_df.loc[i, "emb_P"])
        all_A.append(embed_df.loc[i, "emb_A"])
        all_B.append(embed_df.loc[i, "emb_B"])
        all_dist_PA.append(dist_df.loc[i, "D_PA"])
        all_dist_PB.append(dist_df.loc[i, "D_PB"])
        
        label = embed_df.loc[i, "label"]
        if label == "A": 
            all_label.append(0)
        elif label == "B": 
            all_label.append(1)
        else: 
            all_label.append(2)
    
    return [np.asarray(all_P), np.asarray(all_A), np.asarray(all_B), np.expand_dims(np.asarray(all_dist_PA),axis=1), np.expand_dims(np.asarray(all_dist_PB),axis=1)], all_label

In [12]:
X_train, y_train = create_input(train_emb, train_dist_df)
X_val, y_val = create_input(validation_emb, val_dist_df)

In [13]:
model = End2End_NCR(word_input_shape=X_train[0].shape[1], dist_shape=X_train[3].shape[1]).build()
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, 768)          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 768)          0                                            
__________________________________________________________________________________________________
embedding_

In [14]:
from keras import optimizers
from keras import callbacks

In [15]:
model.compile(optimizer=optimizers.Adam(decay=0.0001), loss="sparse_categorical_crossentropy", metrics=["sparse_categorical_accuracy"])

file_path = "best_model.hdf5"
check_point = callbacks.ModelCheckpoint(file_path, monitor = "val_loss", verbose = 1, save_best_only = True, mode = "min")
early_stop = callbacks.EarlyStopping(monitor = "val_loss", mode = "min", patience=3)
model.fit(X_train, y_train, batch_size=20, epochs=20, validation_data=(X_val, y_val), 
                    shuffle=True, callbacks = [check_point, early_stop])

Train on 2000 samples, validate on 454 samples
Epoch 1/20
2000/2000 [==============================] - 3s 1ms/step - loss: 0.8014 - sparse_categorical_accuracy: 0.6650 - val_loss: 0.7846 - val_sparse_categorical_accuracy: 0.6718

Epoch 00001: val_loss improved from inf to 0.78458, saving model to best_model.hdf5
Epoch 2/20
2000/2000 [==============================] - 1s 444us/step - loss: 0.6835 - sparse_categorical_accuracy: 0.7125 - val_loss: 0.7659 - val_sparse_categorical_accuracy: 0.6718

Epoch 00002: val_loss improved from 0.78458 to 0.76593, saving model to best_model.hdf5
Epoch 3/20
2000/2000 [==============================] - 1s 440us/step - loss: 0.5610 - sparse_categorical_accuracy: 0.7630 - val_loss: 0.7116 - val_sparse_categorical_accuracy: 0.6806

Epoch 00003: val_loss improved from 0.76593 to 0.71157, saving model to best_model.hdf5
Epoch 4/20
2000/2000 [==============================] - 1s 441us/step - loss: 0.4604 - sparse_categorical_accuracy: 0.8060 - val_loss: 0.792

In [16]:
X_test, y_test = create_input(test_emb, test_dist_df)

In [17]:
model.load_weights("./best_model.hdf5")
y_preds = model.predict(X_test, batch_size = 1024, verbose = 1)

sub_df_path = os.path.join('../input/gendered-pronoun-resolution/', 'sample_submission_stage_1.csv')
sub_df = pd.read_csv(sub_df_path)
sub_df.loc[:, 'A'] = pd.Series(y_preds[:, 0])
sub_df.loc[:, 'B'] = pd.Series(y_preds[:, 1])
sub_df.loc[:, 'NEITHER'] = pd.Series(y_preds[:, 2])

sub_df.head(20)

2000/2000 [==============================] - 0s 91us/step


,ID,A,B,NEITHER
0,development-1,0.680997,0.247221,0.071782
1,development-2,0.879528,0.098039,0.022433
2,development-3,0.038694,0.806677,0.154629
3,development-4,0.025057,0.823317,0.151627
4,development-5,0.202918,0.676270,0.120812
5,development-6,0.980097,0.012869,0.007034
6,development-7,0.609203,0.335463,0.055334
7,development-8,0.222455,0.625210,0.152335
8,development-9,0.021900,0.782036,0.196064
9,development-10,0.904015,0.073344,0.022641


In [18]:
from sklearn.metrics import log_loss
y_one_hot = np.zeros((2000, 3))
for i in range(len(y_test)):
    y_one_hot[i, y_test[i]] = 1
log_loss(y_one_hot, y_preds)

0.6622605040314739

In [19]:
sub_df.to_csv("submission.csv", index=False)